## COMSM0022 Computational Logic for Artificial Intelligence Assignment
##### James O'Sullivan, gx19928@bristol.ac.uk, Trisha Khallaghi, trisha.khallaghi@bristol.ac.uk
This Notebook is the report for the Computational Logic for Artificial Intelligence coursework. It will cover our implementation of Negation and Default Rule, with examples provided for each. Due to issues with pip, the implementations were tested utilising an Ubuntu subsystem, with the outputs displayed as code with each section.

For each addition, changes were made to both the <code>prolexa_engine.pl</code> and <code>prolexa_grammar.pl</code>.

## Negation



In this section, we will provide our implementation of Negation in Prolexa, utilising the example provide in the assignment description:

<code>Every teacher is happy. Donald is not happy. Therefore, Donald is not a teacher</code>


### Prolexa Grammar 

To implement Negation, the grammar of Prolexa needs to understand the effect of the word "not" within a sentence. To add this, the first step is to include the not operator in the grammar file.

```julia
:- op(900, fy, not).
```

Additionally, the terms happy, teacher, and Donald were included in the vocabulary of Prolexa as well, using the pre-existing templates provided for proper nouns, nouns, and adjectives:

```julia
pred(teacher, 1,[n/teacher]).
pred(happy,   1,[a/happy]).
proper_noun(s,donald) --> [donald].
stored_rule(1,[(teacher(X):-happy(X))]).
```

Now with the vocabulary added to Prolexa, the next step is to implement verb phrases that include a negated term:

```julia
verb_phrase(s,not(M)) --> [is],[not],property(s,M).
verb_phrase(s,not(M)) --> [are],[not],property(s,M).
verb_phrase(s,not(M)) --> [does],[not],property(s,M).
verb_phrase(s,not(M)) --> [not],property(s,M).
```

With new verb phrases, the definitions of sentences need to be extended to include negated terms. These sentences build upon the grammar rules already present in the grammar engine, including a special case for when the verb phrase itself is negative.

```julia
sentence1([(H:-not(B))]) --> determiner(N,M1,M2,[(H:-B)]),noun(N,M1),verb_phrase(N,not(M2)).
sentence1([(not(L):-true)]) --> proper_noun(N,X),verb_phrase(N,not(X=>L)).
```

The final part of extending the grammar will involve the addition of new question rules which will allow Prolexa to correctly identify questions that contain Negation

```julia
question1(not(Q)) --> [who],verb_phrase(s,not(_X=>Q)).
question1(not(Q)) --> [is],proper_noun(N,X),verb_phrase(N,not(X=>Q)).
```

With these additions to the grammar rules, Prolexa is now capable of recognising negation in sentences and questions, as demonstrated below. However, the engine needs modifying for negations to be saved as knowledge. 

In [ ]:
Hello! I'm ProlexaPlus! Tell me anything, ask me anything.
> donald is happy
*** utterance(donald is happy)
*** rule([(happy(donald):-true)])
*** answer(I will remember that donald is happy)
*** answer(I heard you say,  donald is happy , could you rephrase that please?)
I will remember that donald is happy
> tell me about donald
*** utterance(tell me about donald)
*** goal(all_answers(donald,_2882))
*** answer(donald is a teacher. donald is happy)
donald is a teacher. donald is happy
> donald is not happy
*** utterance(donald is not happy)
*** rule([(not(happy(donald)):-true)])
*** answer(I will remember that donald is not happy)
*** answer(I heard you say,  donald is not happy , could you rephrase that please?)
I will remember that donald is not happy
> tell me about donald
*** utterance(tell me about donald)
*** goal(all_answers(donald,_22684))
*** answer(donald is a teacher. donald is happy)
donald is a teacher. donald is happy

### Prolexa Engine

With <code>prolexa_engine.pl</code> the majority of changes for new features will be made to the meta-interpreter confirmation rules <code>prove_rb</code>. In the case of negation, this can be achieved via the addition of a new rule which utilises proof by contrapositive to handle negation cases in Prolexa

```julia
prove_rb(not(B), Rulebase,P0,P):-
	find_clause((A:-B),Rule,Rulebase),
	prove_rb(not(A),Rulebase,[p(not(B),Rule)|P0],P).
```

This can be defined formally, with A as the antecedent and Q as the consequent of the following relationship:

$$
A \implies B
$$

In this example, if Donald is a teacher, it implies that they are happy. By definition, the contrapositive,

$$
\lnot A \implies \lnot B
$$

is logically equivalent to it i.e., if Donald is a not a teacher, it implies that they are not happy. Therefore, with their logical equivalence, by confirming the second statement we confirm the first, and vice-versa.

Additionally, modifications were made to the <code>prove_question</code> and <code>explain_question</code> predicates to allow for negative queries.

```julia
prove_question(Query,Answer):-
	findall(R,prolexa:stored_rule(_SessionId,R),Rulebase),
	( ...
	; prove_rb(not(Query),Rulebase) ->
		transform(not(Query),Clauses),
		phrase(sentence(Clauses),AnswerAtomList),
		atomics_to_string(AnswerAtomList," ",Answer)
	; Answer = ""
	).

explain_question(Query,SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( ...
	; prove_rb(not(Query),Rulebase,[],Proof) ->
		maplist(pstep2message,Proof,Msg),
		phrase(sentence1([(not(Query):-true)]),L),
		atomic_list_concat([therefore|L]," ",Last),
		append(Msg,[Last],Messages),
		atomic_list_concat(Messages,"; ",Answer)
	; Answer = 'Sorry, I don\'t think this is the case'
	).
```	

In [ ]:
Hello! I'm ProlexaPlus! Tell me anything, ask me anything.
> Donald is not happy
*** utterance(Donald is not happy)
*** rule([(not(happy(donald)):-true)])
*** answer(I will remember that Donald is not happy)
*** answer(I heard you say,  Donald is not happy , could you rephrase that please?)
I will remember that Donald is not happy
> tell me about Donald
*** utterance(tell me about Donald)
*** goal(all_answers(donald,_2882))
*** answer(donald is not happy)
donald is not happy

Another potential approach for implementing negation would be to utilise Modus tolens, which is formalised as follows:

$$
A \implies B, \lnot B \therefore \lnot A
$$

This could be implemented with the following predicate:

```julia
prove_rb(not(A),Rulebase,P0,P):-
	find_clause((B:-A),Rule,Rulebase),
	prove_rb(not(B),Rulebase,[p(A,Rule)|P0],P).
```

## Default Rules


This section will walk through the implementation of Default Rules in Prolexa. Once again, the example provided in the assignment description will be used as the example:

<code>Most birds fly except penguins. Tweety is a bird. Therefore, assuming Tweety is not a penguin, Tweety flies.</code>

### Prolexa Grammar 

For <code>prolexa_grammer.pl</code>, a new determiner was created for the quanitfier "most"

```julia
determiner(p,X=>B,X=>H,X=>E,[(H:-B,not(E))]) --> [most].
```

This determiner allows the addition of an extra noun/proper noun in the sentences for default rules

```julia
sentence1(C) --> determiner(N,M1,M2,M3,C),noun(N,M1),verb_phrase(N,M2),[except],noun(N,M3).
sentence1(C) --> determiner(N,M1,M2,M3,C),noun(N,M1),verb_phrase(N,M2),[except],proper_noun(N,M3).
```

Alongside these new sentence structures, new verb phrases have been added

```julia
verb_phrase(s,not(M)) --> [does,not],iverb(p,M),
verb_phrase(p,not(M)) --> [do,not],iverb(p,M),
```

### Prolexa Engine



The changes to the <code>prolexa_engine.pl</code> are similar to the changes made for Negation, with the <code>prove_rb</code> rules of the meta-interpreter being altered to implement negation as failure from Simply Logical, chapter 3.

```julia
prove_rb(not(A),Rulebase,P0,P):-
    prove_rb(A,Rulebase,P0,P), !, fail.
prove_rb(not(_A),_Rulebase,P,P):-!.
```

In [ ]:
> most birds fly except penguins
*** utterance(most birds fly except penguins)
*** rule([(fly(_3118):-bird(_3118),not(penguin(_3118)))])
*** answer(I will remember that most birds fly except penguins)
*** answer(I heard you say,  most birds fly except penguins , could you rephrase that please?)
I will remember that most birds fly except penguins
> tweety is a penguin
*** utterance(tweety is a penguin)
*** rule([(penguin(tweety):-true)])
*** answer(I will remember that tweety is a penguin)
*** answer(I heard you say,  tweety is a penguin , could you rephrase that please?)
I will remember that tweety is a penguin
> tell me about tweety
*** utterance(tell me about tweety)
*** goal(all_answers(tweety,_29312))
*** answer(tweety is not human. tweety is not mortal. tweety is a bird. tweety is a penguin. tweety is not a sparrow. tweety does not fly. tweety is not a teacher. tweety is not happy)
tweety is not human. tweety is not mortal. tweety is a bird. tweety is a penguin. tweety is not a sparrow. tweety does not fly. tweety is not a teacher. tweety is not happy

## Conclusion

We have demonstrated two potential expansions to Prolexa, negation and default rules, highlighting each approaches shortcomings, and where further work can be undertaken.

On the whole, our implementation of negation is servicable, but lacks the ability to remove conflicting rules within the 